<a href="https://colab.research.google.com/github/FNSY96/TerrorismEventsPrediction/blob/master/Graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# ! git config --global user.email "fadynabilyacoub@gmail.com"
# ! git config --global user.name "FNSY96"

In [0]:
! ls

adc.json  Gecon40_post_final.csv  gtd.csv
drive	  GeoEPR-2018.1.csv	  sample_data


In [0]:
# ! git init
! git commit -m "first commit after uploading gtd"
! git remote add origin https://github.com/FNSY96/TerrorismEventsPrediction.git
! git push -u origin master

Reinitialized existing Git repository in /content/.git/
On branch master

Initial commit

Untracked files:
	.config/
	Gecon40_post_final.csv
	GeoEPR-2018.1.csv
	adc.json
	drive/
	gtd.csv
	sample_data/

nothing added to commit but untracked files present
fatal: remote origin already exists.
error: src refspec master does not match any.
error: failed to push some refs to 'https://github.com/FNSY96/TerrorismEventsPrediction.git'


In [0]:
! git log

fatal: your current branch 'master' does not have any commits yet


# **Python Imports**

In [0]:
import pandas as pd
import csv
import glob
import os
from pandas import ExcelWriter
from pandas import ExcelFile
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io


In [0]:
# !pip3 install pandas

# Data Set Imports

In [0]:
!pip install -U -q PyDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
drive.mount('/content/drive')
datadir  = 'drive/My Drive/GraduationProject/Data'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### **GeoEPR**

In [0]:
fid = drive.ListFile({'q':"title='GeoEPR-2018.1.csv'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('GeoEPR-2018.1.csv')
f.Upload()

### **Gecon40_post_final**

In [0]:
fid = drive.ListFile({'q':"title='Gecon40_post_final.csv'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Gecon40_post_final.csv')
f.Upload()

###**GTD**

In [0]:
!ls drive/'My Drive'/GraduationProject/Data/gtd.csv

# Pandas Data Frames

## **CSV**

In [0]:
gtd_df = pd.read_csv(os.path.join(datadir, 'gtd.csv'),encoding='latin-1')
print(type(gtd_df))
# data.head(n=len(data))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>


In [0]:
gecon_df = pd.read_csv(os.path.join(datadir, 'Gecon40_post_final_kh.csv'),encoding='latin-1')


           AREA             COUNTRY         D1         D2        D3        D4  \
0             0  Bosnia&Herzegovina     0.0000     0.0000    0.0000    0.0000   
1      10.47769              Brazil   252.8497   181.7542  168.6656  156.6020   
2      33.62301              Brazil   639.8335   109.1993  109.1993  109.1993   
3      12.15342              Brazil   565.7923   565.7923  522.7566  522.7566   
4      36.46025              Brazil   604.4861   604.4861  547.9117  547.9117   
5             0              Brazil     0.2873     0.2873    0.2873    0.2873   
6             0              Brazil   520.2774   441.4826  184.3687  184.3687   
7      12.35509              Brazil   384.9028   384.9028  384.9028  341.5250   
8      12.34964              Brazil     0.8852     0.8852    0.8852    0.8852   
9             0            Cameroon   627.4135   627.4135   19.5632   19.5632   
10     361.9061            Cameroon  1085.3410  1085.3410   37.3248    4.7014   
11     7773.164             

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,19,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#geo_epr_df = pd.read_csv("GeoEPR-2018.1.csv", header=None)
# gecon_df = pd.read_csv("Gecon40_post_final.csv", header=None)
# population_density_nasa_df =  pd.read_csv("population_density_nasa.csv", header=None)
 gtd_df =  pd.read_csv("gtd.csv", header=None)
# print(geo_epr_df)